In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import statsmodels.api as sm

In [2]:
path_ref = '..\\data\\refinitiv'

In [3]:
df_sample = pd.read_excel(os.path.join(path_ref,'sample.xlsx'))
df_sentiment = pd.read_csv('sentiment_scores.csv') #drop h&m and dhl
df_similarity = pd.read_csv('similarity_scores.csv')

In [37]:
df_sample[df_sample['company'] == 'rio-tinto']

,company,resource use,emissions,environmental innovation,strategy,average_practice,green_practice,green_communication,discrepancy_index
49,rio-tinto,79.17,83.26,50.35,88.54,70.926667,-0.856063,0.169265,0.681803


In [5]:
df_sentiment

,Unnamed: 0,ESG governance,Emission Reduction,Waste Management,Renewable Energy,Plastics Recyling,Electrical Vehicles,Climate Change,Nature Conservation,Sustainable Transportation,Sustainable Agriculure,Sustainable Innovation,Sustainable Finance,Greenwashing Accusations,final_score
0,rio-tinto,-0.103401,-0.118888,0.315222,-0.076896,0.623388,-0.104418,0.069384,0.360745,-0.046325,NaN,-0.238337,-0.540579,NaN,0.012718
1,walmart,0.007098,0.063356,0.073705,-0.227604,0.030754,-0.283509,0.094747,0.065251,NaN,0.063302,-0.185091,0.188790,NaN,-0.009927
2,colgate,0.114434,0.066575,-0.044937,-0.419891,0.065245,NaN,-0.005104,-0.015416,NaN,-0.323991,NaN,-0.046383,NaN,-0.067719
3,general-mills,-0.062815,0.007971,-0.185231,0.031976,-0.213897,NaN,0.264606,-0.249514,-0.884435,-0.139983,-0.116805,0.105186,NaN,-0.131176
4,hp,-0.080099,-0.158046,-0.027431,0.018113,-0.037755,0.125385,0.041401,-0.157452,0.215793,-0.205014,-0.147086,0.367242,NaN,-0.003746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,adidas,-0.003448,0.293583,0.098622,0.148214,-0.018610,-0.129327,0.029701,0.256966,NaN,NaN,0.084790,0.091009,NaN,0.085150
59,hyundai,0.042043,-0.102316,-0.034380,0.024789,0.216669,0.135543,-0.122541,-0.169336,0.205020,-0.912826,0.157662,NaN,NaN,-0.050879
60,intel,0.125090,0.207181,-0.075009,0.134263,0.077224,0.261944,0.305286,0.153274,-0.133883,NaN,0.158778,-0.428189,NaN,0.071451
61,bmw,-0.234862,0.113722,0.058844,0.042602,NaN,0.206232,0.006581,-0.170647,0.036271,NaN,NaN,NaN,NaN,0.007343


In [16]:
df_similarity[df_similarity['company'] == 'rio-tinto']

,company,0,1,2,3,4,5,6,7,8,10,11,20,25,average,dissimilarity
56,rio-tinto,0.382896,0.504364,0.355998,0.515717,0.092742,0.334281,0.483329,0.232824,0.408051,NaN,0.21438,0.179743,NaN,0.173357,0.826643


In [7]:
df_sentiment.rename(columns = {'Unnamed: 0': 'company'}, inplace = True)
df_similarity.rename(columns = {'Unnamed: 0': 'company'}, inplace = True)

In [8]:
df_sentiment['final_score'] = MinMaxScaler().fit_transform(np.array(df_sentiment['final_score']).reshape(-1,1))

### Preparing the Refinitiv data

In [9]:
df_sample['average_practice'] = df_sample[['resource use', 'emissions', 'environmental innovation']].replace(0, np.nan).mean(axis = 1)

In [10]:
df_sample['green_practice'] = StandardScaler().fit_transform(np.array(df_sample['average_practice']).reshape(-1,1))
df_sample['green_communication'] = StandardScaler().fit_transform(np.array(df_sample['strategy']).reshape(-1,1))
df_sample['discrepancy_index'] = df_sample['green_communication']-df_sample['green_practice']

In [35]:
df_sample['discrepancy_index'] = MinMaxScaler().fit_transform(np.array(df_sample['discrepancy_index']).reshape(-1,1))

In [11]:
df_linreg = pd.merge(df_sentiment, df_sample, how = 'left')
df_linreg

,company,ESG governance,Emission Reduction,Waste Management,Renewable Energy,Plastics Recyling,Electrical Vehicles,Climate Change,Nature Conservation,Sustainable Transportation,...,Greenwashing Accusations,final_score,resource use,emissions,environmental innovation,strategy,average_practice,green_practice,green_communication,discrepancy_index
0,rio-tinto,-0.103401,-0.118888,0.315222,-0.076896,0.623388,-0.104418,0.069384,0.360745,-0.046325,...,NaN,0.378735,79.17,83.26,50.35,88.54,70.926667,-0.856063,0.169265,1.025327
1,walmart,0.007098,0.063356,0.073705,-0.227604,0.030754,-0.283509,0.094747,0.065251,NaN,...,NaN,0.339270,77.73,88.08,76.81,87.01,80.873333,-0.032473,0.061204,0.093677
2,colgate,0.114434,0.066575,-0.044937,-0.419891,0.065245,NaN,-0.005104,-0.015416,NaN,...,NaN,0.238553,99.24,93.57,90.48,97.08,94.430000,1.090026,0.772428,-0.317597
3,general-mills,-0.062815,0.007971,-0.185231,0.031976,-0.213897,NaN,0.264606,-0.249514,-0.884435,...,NaN,0.127961,98.55,78.95,21.53,97.03,66.343333,-1.235565,0.768897,2.004462
4,hp,-0.080099,-0.158046,-0.027431,0.018113,-0.037755,0.125385,0.041401,-0.157452,0.215793,...,NaN,0.350043,96.57,71.78,43.59,97.03,70.646667,-0.879247,0.768897,1.648144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,adidas,-0.003448,0.293583,0.098622,0.148214,-0.018610,-0.129327,0.029701,0.256966,NaN,...,NaN,0.504968,82.77,90.63,80.68,76.89,84.693333,0.283825,-0.653553,-0.937377
59,hyundai,0.042043,-0.102316,-0.034380,0.024789,0.216669,0.135543,-0.122541,-0.169336,0.205020,...,NaN,0.267900,62.39,99.39,36.68,62.18,66.153333,-1.251297,-1.692492,-0.441194
60,intel,0.125090,0.207181,-0.075009,0.134263,0.077224,0.261944,0.305286,0.153274,-0.133883,...,NaN,0.481093,99.75,96.67,75.95,97.03,90.790000,0.788632,0.768897,-0.019735
61,bmw,-0.234862,0.113722,0.058844,0.042602,NaN,0.206232,0.006581,-0.170647,0.036271,...,NaN,0.369368,69.28,89.05,83.25,96.07,80.526667,-0.061178,0.701094,0.762272


In [14]:
df_linreg = pd.merge(df_linreg, df_similarity, how = 'left')

In [15]:
df_linreg

,company,ESG governance,Emission Reduction,Waste Management,Renewable Energy,Plastics Recyling,Electrical Vehicles,Climate Change,Nature Conservation,Sustainable Transportation,...,5,6,7,8,10,11,20,25,average,dissimilarity
0,rio-tinto,-0.103401,-0.118888,0.315222,-0.076896,0.623388,-0.104418,0.069384,0.360745,-0.046325,...,0.334281,0.483329,0.232824,0.408051,NaN,0.214380,0.179743,NaN,0.173357,0.826643
1,walmart,0.007098,0.063356,0.073705,-0.227604,0.030754,-0.283509,0.094747,0.065251,NaN,...,0.182003,0.505852,0.475088,NaN,0.468949,0.243316,0.240421,NaN,0.643641,0.356359
2,colgate,0.114434,0.066575,-0.044937,-0.419891,0.065245,NaN,-0.005104,-0.015416,NaN,...,NaN,0.191898,0.245663,NaN,0.232664,NaN,0.332284,NaN,0.205470,0.794530
3,general-mills,-0.062815,0.007971,-0.185231,0.031976,-0.213897,NaN,0.264606,-0.249514,-0.884435,...,NaN,0.413579,0.391257,0.144090,0.506694,0.159895,0.554802,NaN,0.317077,0.682923
4,hp,-0.080099,-0.158046,-0.027431,0.018113,-0.037755,0.125385,0.041401,-0.157452,0.215793,...,0.467884,0.508401,0.404933,0.341146,0.166339,0.363206,0.189046,NaN,0.543057,0.456943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,adidas,-0.003448,0.293583,0.098622,0.148214,-0.018610,-0.129327,0.029701,0.256966,NaN,...,0.094742,0.376804,0.321279,NaN,NaN,0.212580,0.260169,NaN,0.254580,0.745420
59,hyundai,0.042043,-0.102316,-0.034380,0.024789,0.216669,0.135543,-0.122541,-0.169336,0.205020,...,0.530615,0.330169,0.299412,0.569242,0.100385,0.275194,NaN,NaN,0.492684,0.507316
60,intel,0.125090,0.207181,-0.075009,0.134263,0.077224,0.261944,0.305286,0.153274,-0.133883,...,0.229873,0.454132,0.471610,0.374688,NaN,0.494387,0.330656,NaN,0.624501,0.375499
61,bmw,-0.234862,0.113722,0.058844,0.042602,NaN,0.206232,0.006581,-0.170647,0.036271,...,0.521349,0.579657,0.249247,0.527049,NaN,NaN,NaN,NaN,0.811204,0.188796


In [17]:
df_linreg['discrepancy_index'] = MinMaxScaler().fit_transform(np.array(df_linreg['discrepancy_index']).reshape(-1,1))

In [18]:
df_linreg.set_index('company', inplace = True)

In [25]:
df_linreg

,ESG governance,Emission Reduction,Waste Management,Renewable Energy,Plastics Recyling,Electrical Vehicles,Climate Change,Nature Conservation,Sustainable Transportation,Sustainable Agriculure,...,5,6,7,8,10,11,20,25,average,dissimilarity
company,,,,,,,,,,,,,,,,,,,,,
rio-tinto,-0.103401,-0.118888,0.315222,-0.076896,0.623388,-0.104418,0.069384,0.360745,-0.046325,NaN,...,0.334281,0.483329,0.232824,0.408051,NaN,0.214380,0.179743,NaN,0.173357,0.826643
walmart,0.007098,0.063356,0.073705,-0.227604,0.030754,-0.283509,0.094747,0.065251,NaN,0.063302,...,0.182003,0.505852,0.475088,NaN,0.468949,0.243316,0.240421,NaN,0.643641,0.356359
colgate,0.114434,0.066575,-0.044937,-0.419891,0.065245,NaN,-0.005104,-0.015416,NaN,-0.323991,...,NaN,0.191898,0.245663,NaN,0.232664,NaN,0.332284,NaN,0.205470,0.794530
general-mills,-0.062815,0.007971,-0.185231,0.031976,-0.213897,NaN,0.264606,-0.249514,-0.884435,-0.139983,...,NaN,0.413579,0.391257,0.144090,0.506694,0.159895,0.554802,NaN,0.317077,0.682923
hp,-0.080099,-0.158046,-0.027431,0.018113,-0.037755,0.125385,0.041401,-0.157452,0.215793,-0.205014,...,0.467884,0.508401,0.404933,0.341146,0.166339,0.363206,0.189046,NaN,0.543057,0.456943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
adidas,-0.003448,0.293583,0.098622,0.148214,-0.018610,-0.129327,0.029701,0.256966,NaN,NaN,...,0.094742,0.376804,0.321279,NaN,NaN,0.212580,0.260169,NaN,0.254580,0.745420
hyundai,0.042043,-0.102316,-0.034380,0.024789,0.216669,0.135543,-0.122541,-0.169336,0.205020,-0.912826,...,0.530615,0.330169,0.299412,0.569242,0.100385,0.275194,NaN,NaN,0.492684,0.507316
intel,0.125090,0.207181,-0.075009,0.134263,0.077224,0.261944,0.305286,0.153274,-0.133883,NaN,...,0.229873,0.454132,0.471610,0.374688,NaN,0.494387,0.330656,NaN,0.624501,0.375499


In [32]:
df_sentiment_total = pd.read_csv('sentiment_overall.csv')
df_sentiment_total[df_sentiment_total['company'] == 'rio-tinto']

,company,value
14,rio-tinto,-0.073053


In [51]:
df_similarity_total = pd.read_csv('similarity_overall.csv')

In [30]:
df_linreg = pd.merge(df_linreg, df_sentiment_total, how = 'left', left_on = df_linreg.index, right_on = 'company')

In [59]:
df_linreg = pd.merge(df_similarity_total, df_sample,  how = 'left')

In [60]:
df_linreg[df_linreg['dissimilarity_total'].isnull()]

,company,sim_score,dissimilarity_total,resource use,emissions,environmental innovation,strategy,average_practice,green_practice,green_communication,discrepancy_index


In [34]:
df_linreg[df_linreg['company'] =='rio-tinto']['discrepancy_index']

0    0.681803
Name: discrepancy_index, dtype: float64

In [64]:
df_linreg

,company,sim_score,dissimilarity_total,resource use,emissions,environmental innovation,strategy,average_practice,green_practice,green_communication,discrepancy_index
0,nextera,0.576124,0.423876,93.43,85.69,63.03,87.01,80.716667,-0.045445,0.061204,0.556465
1,chevron,0.781576,0.218424,74.35,86.22,79.38,93.33,79.983333,-0.106166,0.507573,0.625649
2,ralph-lauren,0.315532,0.684468,86.13,82.59,80.68,92.38,83.133333,0.154656,0.440476,0.580910
3,coca-cola,0.608070,0.391930,72.06,42.13,90.52,99.51,68.236667,-1.078796,0.944055,0.817897
4,mitsubishi,0.029763,0.970237,97.50,67.50,83.33,97.05,82.776667,0.125124,0.770310,0.629939
...,...,...,...,...,...,...,...,...,...,...,...
58,shell,0.481304,0.518696,95.39,97.45,79.38,97.68,90.740000,0.784492,0.814805,0.546050
59,tesla,0.545558,0.454442,56.51,68.65,86.92,78.64,70.693333,-0.875383,-0.529953,0.589042
60,totalenergies,0.305087,0.694913,93.91,98.47,79.38,98.13,90.586667,0.771796,0.846588,0.552119
61,colgate,0.510728,0.489272,99.24,93.57,90.48,97.08,94.430000,1.090026,0.772428,0.498584


In [29]:
df_linreg.index

Index(['rio-tinto', 'walmart', 'colgate', 'general-mills', 'hp', 'shell',
       'p&g', 'exxon', 'komatsu', 'bayer', 'inditex', 'google', 'diageo',
       'ibm', 'mcdonald', 'tesla', 'mercedes', 'toyota', 'sonoco', 'h&m',
       'totalenergies', 'chipotle', 'coca-cola', 'eversource', 'honda',
       'renault', 'citi', 'dell', 'beiersdorf', 'pepsico', 'airbus', 'henkel',
       'mitsubishi', 'ralph-lauren', 'dupont', 'tesco', 'hershey', 'nike',
       'boeing', 'abb', 'cemex', 'blackrock', 'ford-motor', 'danone', 'enel',
       'volvo', 'volkswagen', 'nestle', 'linde', 'edp', 'schneider-electric',
       'apple', 'mondelez', 'microsoft', 'nextera', 'veolia', 'chevron', 'dhl',
       'adidas', 'hyundai', 'intel', 'bmw', 'starbucks'],
      dtype='object', name='company')

In [39]:
X = df_linreg['value']
y = df_linreg['discrepancy_index']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:      discrepancy_index   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.5653
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.455
Time:                        21:24:18   Log-Likelihood:                 19.086
No. Observations:                  63   AIC:                            -34.17
Df Residuals:                      61   BIC:                            -29.89
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5379      0.023     23.363      0.0

In [65]:
X = df_linreg['dissimilarity_total']
y = df_linreg['discrepancy_index']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:      discrepancy_index   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.5653
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.455
Time:                        22:01:22   Log-Likelihood:                 19.086
No. Observations:                  63   AIC:                            -34.17
Df Residuals:                      61   BIC:                            -29.89
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.5832    